In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import util

/home/luis/bin/miniconda3/envs/hwnascim/lib/python3.10/site-packages/nni/nas/nn/pytorch/layers.py:94: RuntimeWarning: <class 'torch.nn.parameter.Buffer'> is found to be not a nn.Module, which is unexpected. It means your PyTorch version might not be supported.
  warnings.warn(f'{obj} is found to be not a nn.Module, which is unexpected. '


In [2]:

class CustomModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        # Layers from config
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.conv3 = nn.Conv2d(16, 120, 5)
        self.dropout = nn.Dropout()
        self.fc1 = nn.Linear(120, 84)
        self.fc2 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))          # conv + relu
        x = self.pool(x)                   # maxpool
        x = F.relu(self.conv2(x))          # conv + relu
        x = self.pool(x)                   # maxpool
        x = F.relu(self.conv3(x))          # conv + relu
        x = torch.flatten(x, 1)            # flatten (view)
        x = self.fc1(x)
        x = self.dropout(x)
        x = F.relu(x)
        x = self.fc2(x)               # final fc
        return x

num_classes = 10

In [3]:
layer_config_list = []
layer_config_list.append({'type': 'conv', 'in_channels': 3, 'out_channels': 6, 'kernel_size': 5, 'padding' : 0})
layer_config_list.append({'type': 'relu'})
layer_config_list.append({'type': 'pooling', 'mode': 'MAX', 'kernel_size': 2, 'stride': 2, 'padding' : 0})
layer_config_list.append({'type': 'conv', 'in_channels': 6, 'out_channels': 16, 'kernel_size': 5, 'padding' : 0})
layer_config_list.append({'type': 'relu'})
layer_config_list.append({'type': 'pooling', 'mode': 'MAX', 'kernel_size': 2, 'stride': 2, 'padding' : 0})
layer_config_list.append({'type': 'conv', 'in_channels': 16, 'out_channels': 120, 'kernel_size': 5, 'padding' : 0})
layer_config_list.append({'type': 'relu'})
layer_config_list.append({'type': 'view'})
layer_config_list.append({'type': 'fc', 'in_features': 120, 'out_features': 84})
layer_config_list.append({'type': 'dropout'})
layer_config_list.append({'type': 'relu'})
layer_config_list.append({'type': 'fc', 'in_features': 84, 'out_features': num_classes})
layer_config_list

[{'type': 'conv',
  'in_channels': 3,
  'out_channels': 6,
  'kernel_size': 5,
  'padding': 0},
 {'type': 'relu'},
 {'type': 'pooling',
  'mode': 'MAX',
  'kernel_size': 2,
  'stride': 2,
  'padding': 0},
 {'type': 'conv',
  'in_channels': 6,
  'out_channels': 16,
  'kernel_size': 5,
  'padding': 0},
 {'type': 'relu'},
 {'type': 'pooling',
  'mode': 'MAX',
  'kernel_size': 2,
  'stride': 2,
  'padding': 0},
 {'type': 'conv',
  'in_channels': 16,
  'out_channels': 120,
  'kernel_size': 5,
  'padding': 0},
 {'type': 'relu'},
 {'type': 'view'},
 {'type': 'fc', 'in_features': 120, 'out_features': 84},
 {'type': 'dropout'},
 {'type': 'relu'},
 {'type': 'fc', 'in_features': 84, 'out_features': 10}]

In [4]:
model = CustomModel(num_classes=num_classes)
test = util.trace_model(model=model, input_shape=(1, 1, 28, 28), num_classes=num_classes)
test

[{'type': 'conv',
  'in_channels': 3,
  'out_channels': 6,
  'kernel_size': 5,
  'padding': 0},
 {'type': 'relu'},
 {'type': 'pooling',
  'mode': 'MAX',
  'kernel_size': 2,
  'stride': 2,
  'padding': 0},
 {'type': 'conv',
  'in_channels': 6,
  'out_channels': 16,
  'kernel_size': 5,
  'padding': 0},
 {'type': 'relu'},
 {'type': 'pooling',
  'mode': 'MAX',
  'kernel_size': 2,
  'stride': 2,
  'padding': 0},
 {'type': 'conv',
  'in_channels': 16,
  'out_channels': 120,
  'kernel_size': 5,
  'padding': 0},
 {'type': 'relu'},
 {'type': 'view'},
 {'type': 'fc', 'in_features': 120, 'out_features': 84},
 {'type': 'dropout'},
 {'type': 'relu'},
 {'type': 'fc', 'in_features': 84, 'out_features': 10}]

In [5]:
assert layer_config_list == test